In [1]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import requests
from xml.etree.ElementTree import fromstring, ElementTree
# helpers는 엘라스틱 서치 API를 추상화해 몇 가지 기능을 도와주는 라이브러리이며, bulk API는 helpers에 속한다.
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch('http://localhost:9200/')
docs = [] # 벌크 데이터를 저장할 빈 리스트

In [3]:
for i in range(1, 29):
    iStart = (i - 1) * 1000 + 1
    iEnd = i * 1000
    targetSite = 'http://openapi.seoul.go.kr:8088/746e4968796d6b7639366763624a53/xml/TbPublicWifiInfo/{}/{}/'.format(iStart,
                            iEnd)
    
    response = requests.get(targetSite)
    tree = ElementTree(fromstring(response.text))
    root = tree.getroot()
    
    for row in root.iter('row'):
        gu_nm = row.find('X_SWIFI_WRDOFC').text
        place_nm = row.find('X_SWIFI_MAIN_NM').text
        place_x = row.find('LAT').text
        place_y = row.find('LNT').text
        
        doc = {
            # seoul_wifi2라는 인덱스는 존재하지 않지만 엘라스틱서치 특성성 다이내믹 매핑으로 자동 생성된다.
            '_index': 'seoul_wifi2',
            '_source': {
                'gu_nm': gu_nm,
                'place_nm': place_nm,
                'instl_xy': {
                    'lat': float(place_y),
                    'lon': float(place_x)
                }
            }
        }
        docs.append(doc) # 도큐먼트를 만든 후 docs라는 리스트에 추가한다.
    print('END: {:5d}, {:5d}'.format(iStart, iEnd))
print('END')
response = helpers.bulk(es, docs)

END:     1,  1000
END:  1001,  2000
END:  2001,  3000
END:  3001,  4000
END:  4001,  5000
END:  5001,  6000
END:  6001,  7000
END:  7001,  8000
END:  8001,  9000
END:  9001, 10000
END: 10001, 11000
END: 11001, 12000
END: 12001, 13000
END: 13001, 14000
END: 14001, 15000
END: 15001, 16000
END: 16001, 17000
END: 17001, 18000
END: 18001, 19000
END: 19001, 20000
END: 20001, 21000
END: 21001, 22000
END: 22001, 23000
END: 23001, 24000
END: 24001, 25000
END: 25001, 26000
END: 26001, 27000
END: 27001, 28000
END
